In [1]:
from dataclasses import dataclass
from typing import Optional, List, Set, Tuple, Dict, Any

# ---------- AST node definitions ----------
@dataclass(frozen=True)
class Formula: pass

@dataclass(frozen=True)
class Var(Formula):
    name: str

@dataclass(frozen=True)
class Not(Formula):
    child: Formula

@dataclass(frozen=True)
class And(Formula):
    left: Formula
    right: Formula

@dataclass(frozen=True)
class Or(Formula):
    left: Formula
    right: Formula

@dataclass(frozen=True)
class Imply(Formula):
    antecedent: Formula
    consequent: Formula

def show_ast(f: Formula) -> str:
    if isinstance(f, Var): return f.name
    if isinstance(f, Not): return f"¬({show_ast(f.child)})"
    if isinstance(f, And): return f"({show_ast(f.left)} ∧ {show_ast(f.right)})"
    if isinstance(f, Or):  return f"({show_ast(f.left)} ∨ {show_ast(f.right)})"
    if isinstance(f, Imply): return f"({show_ast(f.antecedent)} → {show_ast(f.consequent)})"
    return "?"

# ---------- Knowledge base ----------
@dataclass
class KB:
    rules: List[Imply]   # rules stored as ASTs
    facts: Set[str]      # atomic facts (names)

# Pull (head, body) from an implication AST; here we only handle Var → Var for simplicity
def rule_head_body(rule: Imply) -> Tuple[Optional[str], Optional[str]]:
    a, c = rule.antecedent, rule.consequent
    if isinstance(a, Var) and isinstance(c, Var):
        return c.name, a.name
    return None, None

# ---------- Forward chaining ----------
def forward_chain(kb: KB) -> Tuple[Set[str], List[str]]:
    derived = set(kb.facts)
    fired_log = []
    changed = True
    while changed:
        changed = False
        for r in kb.rules:
            head, body = rule_head_body(r)
            if head is None:  # skip non-Horn rules in this tiny demo
                continue
            if body in derived and head not in derived:
                derived.add(head)
                fired_log.append(f"Applied {show_ast(r)} because {body} is known → add {head}")
                changed = True
    return derived, fired_log

# ---------- Backward chaining with a proof tree (DFS) ----------
def backward_chain(goal: str, kb: KB, visited: Optional[Set[str]] = None) -> Tuple[bool, Dict[str, Any]]:
    if visited is None: visited = set()
    if goal in kb.facts:
        return True, {"goal": goal, "reason": "fact", "children": []}
    if goal in visited:
        return False, {"goal": goal, "reason": "cycle", "children": []}
    visited.add(goal)

    applicable = []
    for r in kb.rules:
        head, body = rule_head_body(r)
        if head == goal:
            applicable.append((r, body))

    if not applicable:
        return False, {"goal": goal, "reason": "no rule concludes goal", "children": []}

    children = []
    for r, body in applicable:
        ok, subtree = backward_chain(body, kb, visited.copy())
        children.append({"rule": show_ast(r), "subproof": subtree})
        if ok:
            return True, {"goal": goal, "reason": "rule", "children": children}

    return False, {"goal": goal, "reason": "tried rules but failed", "children": children}

def pp_proof(node: Dict[str, Any], indent: int = 0) -> str:
    pad = "  " * indent
    s = f"{pad}- Goal: {node.get('goal','?')}  [{node.get('reason','?')}]"
    for ch in node.get("children", []):
        if "rule" in ch and "subproof" in ch:
            s += f"\n{pad}  uses rule {ch['rule']}"
            s += "\n" + pp_proof(ch["subproof"], indent+2)
        else:
            s += "\n" + pp_proof(ch, indent+1)
    return s

# ---------- DEMO 1: Linear chain ----------
# KB: (Q → R), (P → Q), fact P.  Goal: R
kb_linear = KB(
    rules=[Imply(Var("Q"), Var("R")), Imply(Var("P"), Var("Q"))],
    facts={"P"}
)

print("=== DEMO 1: Linear ===")
print("Rules as AST (static representation):")
for r in kb_linear.rules:
    print("  ", show_ast(r))
print("Facts:", kb_linear.facts)

derived, fired = forward_chain(kb_linear)
print("\nForward chaining fired steps:")
for line in fired:
    print("  ", line)
print("Derived facts:", derived)

ok, proof = backward_chain("R", kb_linear)
print("\nBackward chaining proof tree:")
print(pp_proof(proof))
print("Succeeded?" , ok)

# ---------- DEMO 2: Branching ----------
# KB: (P → Q), (R → Q), facts {P, R}. Goal: Q (two possible rules)
kb_branch = KB(
    rules=[Imply(Var("P"), Var("Q")), Imply(Var("R"), Var("Q"))],
    facts={"P", "R"}
)

print("\n\n=== DEMO 2: Branching ===")
print("Rules as AST (static representation):")
for r in kb_branch.rules:
    print("  ", show_ast(r))
print("Facts:", kb_branch.facts)

derived2, fired2 = forward_chain(kb_branch)
print("\nForward chaining fired steps:")
for line in fired2:
    print("  ", line)
print("Derived facts:", derived2)

ok2, proof2 = backward_chain("Q", kb_branch)
print("\nBackward chaining *branching* proof tree:")
print(pp_proof(proof2))
print("Succeeded?" , ok2)

=== DEMO 1: Linear ===
Rules as AST (static representation):
   (Q → R)
   (P → Q)
Facts: {'P'}

Forward chaining fired steps:
   Applied (P → Q) because P is known → add Q
   Applied (Q → R) because Q is known → add R
Derived facts: {'R', 'P', 'Q'}

Backward chaining proof tree:
- Goal: R  [rule]
  uses rule (Q → R)
    - Goal: Q  [rule]
      uses rule (P → Q)
        - Goal: P  [fact]
Succeeded? True


=== DEMO 2: Branching ===
Rules as AST (static representation):
   (P → Q)
   (R → Q)
Facts: {'R', 'P'}

Forward chaining fired steps:
   Applied (P → Q) because P is known → add Q
Derived facts: {'R', 'P', 'Q'}

Backward chaining *branching* proof tree:
- Goal: Q  [rule]
  uses rule (P → Q)
    - Goal: P  [fact]
Succeeded? True
